<a href="https://colab.research.google.com/github/dukes101/LOM-Analytics/blob/main/TeamPerformanceDash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Read in Data

In [ ]:
## Connect to google drive
from google.colab import drive
drive.mount('/content/drive')

## Libraries
!pip install dash
!pip install plotly
!pip install dash-ag-grid
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px
import dash_ag_grid as dag
import plotly.graph_objects as go

In [4]:
## Year by Year Performance Summary for each team
dfYearbyYear = pd.read_csv('/content/drive/My Drive/dfYearbyYear.csv')
metric_list = dfYearbyYear.columns.to_list()[2:]

## Year by Year Averages for entire league
dfYearbyYearAvg = pd.read_csv('/content/drive/My Drive/dfYearbyYearAvg.csv')

## Easiest/Toughest Opponent for each team
dfEasyToughOpp = pd.read_csv('/content/drive/My Drive/dfVersus.csv')

## List the top 3 best performance metrics for each team
dfTopMetrics = pd.read_csv('/content/drive/My Drive/top_metrics_df.csv')

## Lists the top 3 worst performance metrics for each team
dfWorstMetrics = pd.read_csv('/content/drive/My Drive/worst_metrics_df.csv')

## Performance Summary for each team (Championships, Top 3 Finishes, Playoff Appearances)
dfTeamPerformance = pd.read_csv('/content/drive/My Drive/historicteamperformance.csv')

# Team Performance Dashboard

In [5]:
app = Dash()
app.layout = [

    ## Dashboard Title
    html.H1('Player Performance', style={'color': 'white', 'textAlign': 'center'})
    ## Team Dropdown
    ,dcc.Dropdown(
        options=[{'label': i, 'value': i} for i in dfTeamPerformance['Team'].unique()]
        ,value='Luca Hurst'
        ,id='team-dropdown'
        ,style={'width': '50%'})
    ## Historic Performance Table
    ,dag.AgGrid(
        id='historic-performance-grid'
        ,rowData=dfTeamPerformance.to_dict('records')
        ,columnDefs=[{'field': i} for i in dfTeamPerformance.columns]
        ,style={'width': '50%'}
        #,dashGridOptions={'pagination': True, 'paginationPageSize': 5}
    )
    ## Metric Dropdown
    ,dcc.Dropdown(
        options=[{'label': col, 'value': col} for col in metric_list]
        ,value='Points'
        ,id='metric-dropdown'
        ,style={'width': '50%'})
    ## Dynamic Metric Figure
    ,dcc.Graph(
        figure={}
        ,id='metric-by-year-graph'
        ,style={'width': '50%'}
    )
    ]

## Historic Performance Callback
@callback(
    Output('historic-performance-grid', 'rowData')
    ,Input('team-dropdown', 'value')
)
## Updates table
def update_historic_performance_grid(team):
  return dfTeamPerformance[dfTeamPerformance['Team'] == team].to_dict('records')

## Metric Figure Callback
@callback(
    Output('metric-by-year-graph', 'figure')
    ,Input('team-dropdown', 'value')
    ,Input('metric-dropdown', 'value')
)

## Updates figure based on team and metric selected
def update_metric_by_year_graph(team, metric):

  # Filter on team
  dfYearbyYearTeam = dfYearbyYear[dfYearbyYear['Team'] == team]

  # Create main line graph for selected metric
  fig1 = px.line(dfYearbyYearTeam, x='Year', y=metric, title=f'{metric} by Year for {team}')
  fig1.update_xaxes(type='category')

  # Add a line for the historic averages
  #avg_df = dfYearbyYearAvg[dfYearbyYearAvg['Year'].isin(dfYearbyYearTeam['Year'])]
  #fig2 = px.line(avg_df, x='Year', y=metric, line_dash='dash', line_color='red')

  #fig = go.Figure(data=fig1.data + fig2.data)
  #fig.update_layout(title=f'{metric} by Year for {team}')
  return fig1

if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>